In [15]:
# parameters
imgDim = [101, 101]

In [16]:
from keras.applications import mobilenet_v2
model = mobilenet_v2.MobileNetV2(weights=None, input_shape=[*imgDim, 2], classes=17)
print(model.summary())

][0]          
__________________________________________________________________________________________________
block_6_depthwise_relu (ReLU)   (None, 7, 7, 192)    0           block_6_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_6_project (Conv2D)        (None, 7, 7, 64)     12288       block_6_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_6_project_BN (BatchNormal (None, 7, 7, 64)     256         block_6_project[0][0]            
__________________________________________________________________________________________________
block_7_expand (Conv2D)         (None, 7, 7, 384)    24576       block_6_project_BN[0][0]         
__________________________________________________________________________________________________
block_7_expand_BN (BatchNormali (None, 7, 7, 384)    1536        block_7_expand[0][0]         

In [17]:
from keras import backend as K

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [18]:
from keras.optimizers import Adadelta
model.compile(loss='categorical_crossentropy', metrics=[f1], optimizer=Adadelta())

In [19]:
from dataGenerator import DataGenerator
from personal_data import location_data
dataGenerator_train = DataGenerator(location_data, batch_size=128, inputDim=imgDim)
dataGenerator_val = DataGenerator(location_data, batch_size=128, inputDim=imgDim, isTrainingData=False)

In [20]:
from keras import callbacks
import time

# tensorboard --logdir ~/cytodata2019/tensorboard &
tensorboard = callbacks.TensorBoard(update_freq='batch',
    log_dir='../tensorboard/' + time.strftime("%Y%m%d-%H%M%S"))

In [21]:
model.fit_generator(dataGenerator_train.generate(),
    validation_data=dataGenerator_val.generate(),
    callbacks=[tensorboard],
    steps_per_epoch=len(dataGenerator_train),
    validation_steps=len(dataGenerator_val),
    verbose=1, epochs=100)

Epoch 1/100
78/78 [==============================] - 49s 624ms/step - loss: 2.7858 - f1: 0.0298 - val_loss: 4.2126 - val_f1: 0.0457
Epoch 2/100
78/78 [==============================] - 42s 541ms/step - loss: 2.5963 - f1: 0.0493 - val_loss: 5.2164 - val_f1: 0.0000e+00
Epoch 3/100
78/78 [==============================] - 46s 584ms/step - loss: 2.4824 - f1: 0.0820 - val_loss: 5.2417 - val_f1: 0.0081
Epoch 4/100
78/78 [==============================] - 50s 638ms/step - loss: 2.4186 - f1: 0.1114 - val_loss: 5.8271 - val_f1: 0.0634
Epoch 5/100
78/78 [==============================] - 50s 638ms/step - loss: 2.3842 - f1: 0.1300 - val_loss: 6.4024 - val_f1: 0.0347
Epoch 6/100
78/78 [==============================] - 50s 638ms/step - loss: 2.3381 - f1: 0.1408 - val_loss: 3.8809 - val_f1: 0.0780
Epoch 7/100
78/78 [==============================] - 50s 639ms/step - loss: 2.2877 - f1: 0.1778 - val_loss: 5.3448 - val_f1: 0.1231
Epoch 8/100
78/78 [==============================] - 51s 649ms/step - lo

In [ ]:
from storingModel import save_model
save_model(model, mobileNetV2)